**Table of contents**<a id='toc0_'></a>    
- [Mise en place](#toc1_)    
  - [Fonctions et classes utiles](#toc1_1_)    
  - [Récupération des données](#toc1_2_)    
- [Nettoyage des données](#toc2_)    
  - [Fonctions de nettoyage](#toc2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Mise en place](#toc0_)

In [118]:
import numpy as np
import pandas as pd
import re

# for web scraping
import urllib.request, json 
from html.parser import HTMLParser

## <a id='toc1_1_'></a>[Fonctions et classes utiles](#toc0_)

In [119]:
class LangParser(HTMLParser):
    """Parse language names from Wikipedia extracted HTML"""
    def __init__(self):
        HTMLParser.__init__(self)
        self.recording = 0
        self.data = set()

    def handle_starttag(self, tag, attrs):
        if tag == "a":
            self.recording = 1

    def handle_endtag(self, tag):
        if tag == "a":
            self.recording = 0

    def handle_data(self, data):
        if self.recording:
            # avoid adding edit links
            if data.strip() != "edit":
                # remove start & end white spaces + convert to lower case
                data = data.strip().lower()
                # remove everything between parentheses
                data = re.sub(r'\([^)]*\)', '', data)
                self.data.add(data)


def clean_string(string):
    """Return a cleaned version of an input string"""
    # remove code tags
    string = re.sub(r"<code>.*?<\/code>", "", string, flags=re.S)
    # remove img tags
    string = re.sub(r"<img.*?>", "", string)
    # remove all html tags
    string = re.sub(r"<.*?>", "", string)
    # remove newlines
    string = re.sub(r"\n", " ", string)
    # lowercase
    string = string.lower()
    # keep only letters, digits, spaces and some useful characters
    string = re.sub(r"[^\w _.-]", "", string)
    # remove multiple spaces
    string = re.sub(r" +", " ", string)

    return string


## <a id='toc1_2_'></a>[Récupération des données](#toc0_)

[Requête SQL data.stackexchange.com ici](https://data.stackexchange.com/stackoverflow/query/1824062/top-50k-relevant-questions)

⇒ compromis fait entre qualité maximale et contraintes (≥ 5 tags)
- pour qualité, score ≥ 10 et > 3 réponses

🚧 >=1 réponse : voir cohérence & qualité

- dates jusqu’à 2009, cohérent
- reste à spectre large pour tout type de question

seules features indispensables & RGPD dans la requête

In [120]:
df_raw = pd.read_csv("data/2024-03-11 QueryResults.csv")

# <a id='toc2_'></a>[Nettoyage des données](#toc0_)

## Liste des langages de programmation

In [121]:
with urllib.request.urlopen("https://en.wikipedia.org/w/api.php?action=parse&page=List_of_programming_languages&prop=text&format=json&disabletoc=1&formatversion=2") as url:
    scrap = json.load(url)["parse"]["text"]

# get all programming languages ever declared on Wikipedia
start = scrap.find('<h2><span class="mw-headline" id="A')
end = scrap.find('<h2><span class="mw-headline" id="See_also')
scrap = scrap[start:end]

In [145]:
parser = LangParser()
parser.feed(scrap)
prog_langs = parser.data
print(f"Found {len(prog_langs)} programming languages on Wikipedia")

Found 693 programming languages on Wikipedia


## xxxxxxxxxxxxxxx

In [124]:
# remove useless features
df_raw.drop(columns=["Score", "AnswerCount", "CreationDate"], inplace=True)
# change Tags string to list
df_raw["Tags"] = df_raw["Tags"].apply(lambda x: x[1:-2].split("><"))

🚧 Get an extract

In [125]:
ex_body = df_raw["Body"][0]

In [126]:
df_raw

,Title,Body,Tags
0,std::shared_mutex::unlock_shared() blocks even...,<p>My team has encountered a deadlock that I s...,"[c++, windows, multithreading, stl, shared-loc]"
1,"What is the correct output of sizeof(""string"")?","<p>On a microcontroller, in order to avoid loa...","[c, language-lawyer, sizeof, string-literals, ..."
2,Problem loading external scripts like jQuery,<p>I'm facing a problem since this morning wit...,"[javascript, html, jquery, google-apps-script,..."
3,Does a const char* literal string persistently...,<p>I have functions like the following:</p>\n<...,"[c++, constants, constexpr, lifetime, null-ter..."
4,WillPopScope is deprecated in Flutter,<p>'WillPopScope' is deprecated and shouldn't ...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,The activator for bundle is invalid,<p>I'm trying to create a simple plugin in ecl...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,Pre-allocate space for C++ STL queue,<p>I'm writing a radix sort algorithm using qu...,"[c++, performance, memory, stl, queu]"
49997,Adjusting the positions of the labels in a UIT...,<p>I'm using a UITableViewCell with UITableVie...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,Padding on StackPanel?,<p>I am trying to set padding of a StackPanel ...,"[wpf, xaml, layout, padding, stackpane]"


🚧 [HTML Parser](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list) : (cf. [question SoF](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list))  
[RegEx JL](https://www.notion.so/julmat/RegEx-expressions-r-guli-res-70fbbcb177ee476ba9a5ae011d14fe6f) & [RegEx 101](https://regex101.com/?flavor=python&flags=gm)

- suppr chiffres et caractères **seuls** " 123 " ou "."
- suppr certains mots d'1 caractère (sauf si dans langages de programmation)
- suppr mots 1 alpha + 1 chiffre
- suppr points à la fin d'un mot
- ne pas supprimer "+", "#" et autres caractères utiles (noms de langages)

In [127]:
df_raw["Body"] = df_raw["Body"].apply(clean_string)
df_raw["Title"] = df_raw["Title"].apply(clean_string)

In [128]:
print(ex_body)
print(df_raw["Body"][0])

<p>My team has encountered a deadlock that I suspect is a bug in the Windows implementation of SRW locks. The code below is a distilled version of real code. Here's the summary:</p>
<ol>
<li>Main thread acquires exclusive lock</li>
<li>Main thread creates N children threads</li>
<li>Each child thread
<ol>
<li>Acquires a shared lock</li>
<li>Spins until all children have acquired a shared lock</li>
<li>Releases the shared lock</li>
</ol>
</li>
<li>Main thread releases exclusive lock</li>
</ol>
<p>Yes this could be done with std::latch in C++20. That's not the point.</p>
<p>This code works <em>most</em> of the time. However roughly 1 in 5000 loops it deadlocks. When it deadlocks exactly 1 child successfully acquires a shared lock and N-1 children are stuck in <code>lock_shared()</code>. On Windows this function calls into <code>RtlAcquireSRWLockShared</code> and blocks in <code>NtWaitForAlertByThreadId</code>.</p>
<p>The behavior is observed when used <code>std::shared_mutex</code> direc

In [129]:
display(*df_raw["Title"][:20].values)

'stdshared_mutexunlock_shared blocks even though there are no active exclusive locks on windows'

'what is the correct output of sizeofstring'

'problem loading external scripts like jquery'

'does a const char literal string persistently exist as long as the process alive'

'willpopscope is deprecated in flutter'

...

'compilation error after upgrading to jdk 21 - nosuchfielderror jcimport does not have member field jctree qualid'

'ld warning ignoring duplicate libraries -lgcc after the recent update of xcodebuild tools'

'error encountered only below ios 17 devices thread 1 exc_bad_access code1 address0x0'

'handling cycle inside runner building could produce unreliable results after update xcode to xcode 15'

'what is the purpose of _t aliases and _v variable templates for type traits'